In [244]:
import pandas as pd
import pickle
import re
# ^^^ pyforest auto-imports - don't write above this line

# Imports

## Libraries

In [4]:
import folium
import unidecode
import warnings
warnings.filterwarnings('ignore')

## Demonym Dictionary

In [6]:
with open("./my_saved_data/demonym_dictionary.pickle", "rb+") as f:
    demonym_dictionary = pickle.load(f)

# Adding City (Restaurant Inspection) Data
cities with most mexican immigrants in us: https://247wallst.com/economy/2017/01/27/us-cities-with-the-most-mexican-immigrants/

## NYC

In [7]:
rest_insp = pd.read_csv("./food_inspections_data/DOHMH_New_York_City_Restaurant_Inspection_Results.csv")

In [9]:
rest_insp['CUISINE DESCRIPTION'].nunique()

84

In [10]:
# tex-mex, mexican, latin
mex_rest_insp = rest_insp[rest_insp['CUISINE DESCRIPTION'] == 'Mexican']

In [11]:
mex_rest_insp.shape

(16656, 26)

In [12]:
mex_rest_insp.head(2)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
24,41571476,NATALIA BAR,Queens,62-10,39 AVENUE,11377.0,3477065942,Mexican,05/04/2018,Violations were cited in the following area(s).,...,06/14/2020,Cycle Inspection / Re-inspection,40.746844,-73.901406,402.0,26.0,26100.0,4028392.0,4.012320e+09,QN63
27,40996642,RECUERDOS MEXICANOS,Staten Island,232,PORT RICHMOND AVENUE,10302.0,7188155533,Mexican,10/22/2019,Violations were cited in the following area(s).,...,06/14/2020,Cycle Inspection / Initial Inspection,40.635784,-74.134935,501.0,49.0,20700.0,5025513.0,5.010820e+09,SI28


### Removing duplicate restaurants from nyc mexican restaurants df

In [15]:
unique_nyc_mex = mex_rest_insp.drop_duplicates(subset=['DBA'], keep='first')
unique_nyc_mex.reset_index(inplace=True, drop=True)
unique_nyc_mex.shape

(849, 26)

In [16]:
# a 95% reduction of the df!
unique_nyc_mex.shape[0] / mex_rest_insp.shape[0]

0.050972622478386166

In [17]:
# list of restaurant names to compare to values in demonym_dictionary
nyc_rest_insp_names = [x.lower() for x in unique_nyc_mex['DBA']]

### Getting matches 

In [18]:
rest_matches_nyc = []
# list of not-quite-real matches
bad_matches = ['mexican', 'tequila', 'margarita', 'margaritas', 'mexico', 'pedro', 'azteca', 'paraiso'
'mexico','rodeo','rio','maria','mexicanos','coyote','marcos','mama','bravo','viejo','perla','nuevo'
'verde', 'gonzalez', 'corona', 'armadillo', 'arriba', 'palmas', 'delicias', 'blanco', 'crespo','tortuga']
for item in nyc_rest_insp_names:
    split_item = item.split()
    for word in split_item:
        for key, value in demonym_dictionary.items():
            for city in value:
                if word == city:
                    if word not in bad_matches: # if not, the number of results is tripled
                        rest_matches_nyc.append(item)

In [19]:
len(rest_matches_nyc)

259

In [20]:
unique_rest_matches_nyc = set(rest_matches_nyc)

In [21]:
# was 44 before additions to demonym dict ... was 169 before word corrections
len(unique_rest_matches_nyc)

111

### Getting Regions for Matches

In [22]:
restaurant_matches_nyc = {key: None for key in set(unique_rest_matches_nyc)}

In [23]:
for key, value in restaurant_matches_nyc.items():
    split_item = key.split()
    for word in split_item:
        for key1, value1 in demonym_dictionary.items():
            for city in value1:
                if word == city:
                    value_info = [key1, city]
                    restaurant_matches_nyc[key] = value_info

In [24]:
len(restaurant_matches_nyc)

111

#### Removing all entries that passed filter incorrectly

In [25]:
bad_keys = []
bad_matches = ['mexican','mexico','mexicanos','azteca','esperanza','estrada','esperanzas','salero','pinos',
                  'maria', 'bravo', 'nuevo', 'progreso', 'delicias', 'comales', 'palmas', 'palenque', 'concordia',
                 'china', 'gym', 'paloma', 'rio', 'mex', 'tequila', 'colorado', 'ventana', 'lom', 'garcia', 'paz',
                 'chavez', 'paraiso', 'senor', 'oriental', 'fronteras', 'tap', 'aca', 'purisima', 'rodriguez',
                 'hernandez', 'sanchez', 'victoria', 'oasis', 'cash', 'pinas', 'yaa', 'tap', 'meson','agustin',
                 'agustin', 'limon', 'alamo', 'slp', 'providencia', 'reyes', 'lom', 'verde', 'perla', 'madrid', 
                 'delta', 'mama', 'lopez', 'honey', 'laurel', 'california pizza kitchen', 'sauces', 'laguna', 
              'dolores', 'presidio', 'ver', 'bernal', 'rincon', 'marin', 'palma', 'potrero', 'mid', 'valencia',
              'aura', 'kava', 'pueblito', 'castillo', 'tam', 'marcos', 'montecristo', 'tinajas', 'alvarado',
              'porvenir', 'nieves', 'mina', 'marin', "alamos", 'reforma', 'jal', 'margaritas', 'california']
for key, value in restaurant_matches_nyc.items():
    if value[1] in bad_matches:
        bad_keys.append(key)

In [26]:
len(bad_keys)

25

In [27]:
for x in bad_keys:
    if restaurant_matches_nyc.get(x) != None:
        del restaurant_matches_nyc[x]

In [28]:
len(restaurant_matches_nyc)

86

In [29]:
list(restaurant_matches_nyc.items())[:2]

[('la guadalupe restaurant', ['Chihuahua', 'guadalupe']),
 ('tacos cuautla morales', ['Veracruz', 'morales'])]

### Getting coordinates for each restaurant

In [30]:
region_in_name = []
for x in unique_nyc_mex['DBA']:
    if x.lower() in restaurant_matches_nyc.keys():
        region_in_name.append(1)
    else:
        region_in_name.append(0)

In [31]:
unique_nyc_mex['region_in_name'] = region_in_name

In [34]:
nyc_mex_rest_w_region = unique_nyc_mex[unique_nyc_mex['region_in_name'] == 1]
nyc_mex_rest_w_region.reset_index(inplace=True, drop=True)
nyc_mex_rest_w_region.shape

(86, 27)

In [1095]:
nyc_mex_rest_w_region.DBA.nunique()

86

In [35]:
nyc_mex_rest_w_region.head(2)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,region_in_name
0,40909425,PUEBLA SEAFOOD,Queens,95-27,ROOSEVELT AVENUE,11372.0,7186723556,Mexican,04/19/2019,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.749015,-73.870844,403.0,21.0,27300.0,4036619.0,4.014830e+09,QN28,1
1,41292582,TACOS PUEBLA RESTAURANT,Bronx,2181,GRAND CONCOURSE,10453.0,7182205463,Mexican,08/13/2019,Violations were cited in the following area(s).,...,Cycle Inspection / Re-inspection,40.855330,-73.901288,205.0,14.0,23704.0,2013760.0,2.031620e+09,BX40,1


### Count of regions represented in nyc mexican restaurant names

In [36]:
regions_represented = []
for key, value in restaurant_matches_nyc.items():
    if value[0] not in regions_represented:
        regions_represented.append(value[0])
regions_represented[0:3]

['Chihuahua', 'Veracruz', 'Tamaulipas']

In [40]:
region_list = []
for x in nyc_mex_rest_w_region['DBA']:
    region_list.append(restaurant_matches_nyc[x.lower()][0])
len(region_list)

86

In [39]:
nyc_mex_rest_w_region['region'] = region_list

In [41]:
# i get a lot of repeats with this method 
nyc_mex_rest_w_region['region'].value_counts()

Puebla                 32
Yucatan                11
Chihuahua               6
Baja California         6
Morelos                 4
Oaxaca                  4
Veracruz                3
Tamaulipas              3
Hidalgo                 2
Quintana Roo            2
Guerrero                2
Jalisco                 2
Mexico                  1
Baja California Sur     1
Sinaloa                 1
Sonora                  1
Colima                  1
Tabasco                 1
Aguascalientes          1
Tlaxcala                1
Durango                 1
Name: region, dtype: int64

### Getting region counts as a percentage

In [42]:
nyc_region_counts_series = nyc_mex_rest_w_region['region'].value_counts()

In [43]:
nyc_region_counts_dict = nyc_region_counts_series.to_dict()

In [45]:
nyc_region_counts_dict_pct = {key: round(value/len(region_list), 2) 
                              for key, value in nyc_region_counts_dict.items()}

In [46]:
list(nyc_region_counts_dict_pct.items())[0:2]

[('Puebla', 0.37), ('Yucatan', 0.13)]

### Getting Values that matches

In [47]:
nyc_values = {value[1]:0 for key, value in restaurant_matches_nyc.items()}

In [48]:
for key, value in restaurant_matches_nyc.items():
    if value[1] in nyc_values.keys():
        nyc_values[value[1]] += 1

In [49]:
# https://careerkarma.com/blog/python-sort-a-dictionary-by-value/
sort_values = sorted(nyc_values.items(), key=lambda x: x[1], reverse=True)
for i in sort_values[0:5]:
    print(i[0], i[1])

puebla 9
tulcingo 5
poblana 5
guadalupe 4
oaxaca 4


### Adding to base_map

In [50]:
base_lat = 40.7128
base_long = -74.0060
base_map = folium.Map([base_lat, base_long], zoom_start=10, tiles="cartodbpositron")

In [51]:
len(regions_represented)

21

In [52]:
# a color for each region
color_list = ["red" 'blue', 'green', 'purple', 'orange', 'darkred',
'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple']
#white’, ‘pink’, ‘lightblue’, ‘lightgreen’, ‘gray’, ‘black’, ‘lightgray’]
len(color_list)

11

In [53]:
for x in range(nyc_mex_rest_w_region.shape[0]):
    rest_name = nyc_mex_rest_w_region['DBA'][x].lower()
    region_city_name = restaurant_matches_nyc[rest_name]
    color_list_index = regions_represented.index(region_city_name[0])
    if color_list_index >= 10: # to account for the most represented regions and use the colors available
        color_list_index = 10
    folium.Marker(
            location = (nyc_mex_rest_w_region['Latitude'][x], nyc_mex_rest_w_region['Longitude'][x]),
            popup = f"name '{rest_name}', region '{region_city_name[0]}'",
            icon=folium.Icon(color=color_list[color_list_index])
        ).add_to(base_map)

In [54]:
base_map

### Saving the map

In [1119]:
# base_map.save("mexican_restaurants_in_new_york_city_with_region_in_name.html")

### Map Legend

In [55]:
region_legend = []
for key, value in restaurant_matches_nyc.items():
    if value[0] not in region_legend:
        region_legend.append(value[0])

In [56]:
full_legend = list(zip(region_legend, color_list[:-1]))

In [57]:
# all other regions are "darkpurple"
full_legend

[('Chihuahua', 'redblue'),
 ('Veracruz', 'green'),
 ('Tamaulipas', 'purple'),
 ('Colima', 'orange'),
 ('Puebla', 'darkred'),
 ('Oaxaca', 'lightred'),
 ('Baja California', 'beige'),
 ('Morelos', 'darkblue'),
 ('Tlaxcala', 'darkgreen'),
 ('Jalisco', 'cadetblue')]

### Turning dictionary to df

In [58]:
rest_df = pd.DataFrame.from_dict(restaurant_matches_nyc, orient="index")
rest_df.reset_index(inplace=True)
rest_df.columns = ['Restaurant_name', 'Region', 'Matching_word_w_region']

In [59]:
rest_df.head()

,Restaurant_name,Region,Matching_word_w_region
0,la guadalupe restaurant,Chihuahua,guadalupe
1,tacos cuautla morales,Veracruz,morales
2,acapulco deli & restaurant,Tamaulipas,acapulco
3,rinconcito tepeyac,Colima,tepeyac
4,tulcingo restaurant,Puebla,tulcingo


In [60]:
rest_df.columns

Index(['Restaurant_name', 'Region', 'Matching_word_w_region'], dtype='object')

#### Checking to see if "Tulcingo Deli Grocery" is in database

In [61]:
'TULCINGO' in list(nyc_mex_rest_w_region['DBA'].unique())

True

In [65]:
tulcingo_count = 0
for x in rest_df.Matching_word_w_region:
    if x.lower() == 'tulcingo':
        tulcingo_count += 1
tulcingo_count

5

## LA

In [66]:
LA_data = pd.read_csv("./food_inspections_data/Map_of_Restaurants_LA.csv")
LA_data.shape

(7017, 16)

In [67]:
LA_data.head(2)

,LOCATION ACCOUNT #,BUSINESS NAME,DBA NAME,STREET ADDRESS,CITY,ZIP CODE,LOCATION DESCRIPTION,MAILING ADDRESS,MAILING CITY,MAILING ZIP CODE,NAICS,PRIMARY NAICS DESCRIPTION,COUNCIL DISTRICT,LOCATION START DATE,LOCATION END DATE,LOCATION
0,0003030535-0001-6,"NIGHT MARKET 3, INC.",NaN,2533 LINCOLN BLVD,VENICE,90291-5042,2533 LINCOLN 90291,2533 LINCOLN BLVD,VENICE,90291-5042,722110,Full-service restaurants,11,02/01/2018,NaN,"(33.9915, -118.449)"
1,0002893692-0002-0,NORMA L DIAZ SANCHEZ,PLAYAS EL SALVADOR RESRAURANTE,4052 1/2 S CENTRAL AVENUE,LOS ANGELES,90011-2866,4052 Central 90011-2866,NaN,NaN,NaN,722110,Full-service restaurants,9,05/19/2020,NaN,"(34.0101, -118.2563)"


In [68]:
LA_data['LOCATION'].fillna("None", inplace=True)

In [69]:
LA_data['LOCATION'].dtype

dtype('O')

### Making sure that there aren't any duplicate restaurants

In [70]:
len(LA_data['BUSINESS NAME']) == len(set(LA_data['BUSINESS NAME']))

False

In [71]:
len(LA_data['BUSINESS NAME'])

7017

In [72]:
len(set(LA_data['BUSINESS NAME']))

6125

In [73]:
unique_LA_data = LA_data.drop_duplicates(subset=['DBA NAME'], keep='first')

### Adding each name to a new column in LA_data

In [74]:
possible_biz_names = list(zip(unique_LA_data['BUSINESS NAME'],unique_LA_data['DBA NAME']))

In [75]:
list_possible_biz_names = []
for x in possible_biz_names:
    y = list(x)
    if np.nan in y:
        y.remove(np.nan)
    lower_item_list = [item.lower() for item in y]        
    list_possible_biz_names.append(lower_item_list)

In [76]:
unique_LA_data['biz_names'] = list_possible_biz_names

### Getting Matches

In [78]:
unique_LA_data['biz_names'][0:4]

0                               [night market 3, inc.]
1    [norma l diaz sanchez, playas el salvador resr...
3        [bombay curry inc, bombay curry and pizzeria]
4    [t & m finatic enterprise inc, sharky's fresh ...
Name: biz_names, dtype: object

In [79]:
len(unique_LA_data['biz_names'][1])

2

In [80]:
rest_matches_la = []
# list of not-quite-real matches
bad_matches = ['mexican', 'tequila', 'margarita', 'margaritas', 'mexico', 'pedro', 'azteca', 'paraiso'
'mexico','rodeo','rio','maria','mexicanos','coyote','marcos','mama','bravo','viejo','perla','nuevo'
'verde', 'gonzalez', 'corona', 'armadillo', 'arriba', 'palmas', 'delicias', 'blanco', 'crespo','tortuga']
for item in unique_LA_data['biz_names']:
    for word in item:
        for key, value in demonym_dictionary.items():
            for city in value:
                if word == city:
                    if word not in bad_matches: # if not, the number of results is tripled
                        rest_matches_la.append(item)

In [83]:
unique_rest_matches_la = []
for x in rest_matches_la:
    if x not in unique_rest_matches_la:
        unique_rest_matches_la.append(x)
len(unique_rest_matches_la)

12

In [84]:
# most of this data looks like it's incorrectly matching so I will not use LA's data
unique_rest_matches_la[0:4]

[['luis e ponce', 'los pilares'],
 ['rio nuevo, llc', 'salazar'],
 ['toshihiko t hoshi', 'toshi'],
 ['genoveva/guillermo padilla', 'la barca']]

## Chicago 

In [85]:
chi_data = pd.read_csv("./food_inspections_data/Food_Inspections_chicago.csv")
chi_data.shape

(206916, 17)

In [86]:
chi_data.head(2)

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2373714,CHIYA CHAI CAFE,CHIYA CHAI CAFE,2432644.0,Restaurant,Risk 1 (High),2770 N MILWAUKEE AVE,CHICAGO,IL,60647.0,06/15/2020,Canvass Re-Inspection,Pass,NaN,41.931449,-87.711547,"(-87.71154708820332, 41.93144884388019)"
1,2373680,TROPI CUBA,TROPI CUBA,1422721.0,Restaurant,Risk 1 (High),3000 W LYNDALE ST,CHICAGO,IL,60647.0,06/12/2020,Canvass Re-Inspection,Pass,47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE...,41.922493,-87.702359,"(-87.70235877406888, 41.92249266994513)"


In [87]:
chi_rest = chi_data[chi_data['Facility Type'] == 'Restaurant']
chi_rest.shape

(137853, 17)

In [88]:
# reduce the size of df by 33% 
chi_rest.shape[0] / chi_data.shape[0] 

0.6662268746737806

In [89]:
chi_rest.head(1)

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2373714,CHIYA CHAI CAFE,CHIYA CHAI CAFE,2432644.0,Restaurant,Risk 1 (High),2770 N MILWAUKEE AVE,CHICAGO,IL,60647.0,06/15/2020,Canvass Re-Inspection,Pass,NaN,41.931449,-87.711547,"(-87.71154708820332, 41.93144884388019)"


### Getting Names of restaurants

In [90]:
possible_chi_biz_names = list(zip(chi_rest['DBA Name'], chi_rest['AKA Name']))

In [91]:
chi_biz_names = []
for x in possible_chi_biz_names:
    x_list = list(x)
    if x_list[0] == x_list[1]:
        del x_list[1]
    elif np.nan in x_list:
        x_list.remove(np.nan)
    lower_possible_chi_biz_names = [item.lower() for item in x_list]
    chi_biz_names.append(lower_possible_chi_biz_names)

In [92]:
chi_biz_names[0:3]

[['chiya chai cafe'], ['tropi cuba'], ['china gourmet']]

In [93]:
chi_rest['biz_names'] = chi_biz_names

### Getting Matches with Dem Dict

In [94]:
rest_matches_chi = []
# list of not-quite-real matches
bad_matches = ['mexican','mexico','mexicanos','azteca','esperanza','estrada','esperanzas','salero','pinos',
                  'maria', 'bravo', 'nuevo', 'progreso', 'delicias', 'comales', 'palmas', 'palenque', 'concordia',
                 'china', 'gym', 'paloma', 'rio', 'mex', 'tequila', 'colorado', 'ventana', 'lom', 'garcia', 'paz',
                 'chavez', 'paraiso', 'senor', 'oriental', 'fronteras', 'tap', 'aca', 'purisima', 'rodriguez',
                 'hernandez', 'sanchez', 'victoria', 'oasis', 'cash', 'pinas', 'yaa', 'tap', 'meson','agustin',
                 'agustin', 'limon', 'alamo', 'slp', 'providencia', 'reyes', 'lom', 'verde', 'perla', 'madrid', 
                 'delta', 'mama', 'lopez', 'honey', 'laurel', 'california pizza kitchen', 'sauces', 'laguna', 
              'dolores', 'presidio', 'ver', 'bernal', 'rincon', 'marin', 'palma', 'potrero', 'mid', 'valencia',
              'aura', 'kava', 'pueblito', 'castillo', 'tam', 'marcos', 'montecristo', 'tinajas', 'alvarado',
              'porvenir', 'nieves', 'mina', 'marin']
for item in chi_rest['biz_names']:
    for word in item:
        for key, value in demonym_dictionary.items():
            for city in value:
                if word == city:
                    if word not in bad_matches: # if not, the number of results is tripled
                        rest_matches_chi.append(item)

In [95]:
len(rest_matches_chi)

765

In [96]:
rest_matches_chi[0:5]

[['los comales'],
 ['la amistad inc.', 'la amistad'],
 ['las palmas'],
 ['las palmas'],
 ['la fuente']]

In [100]:
unique_rest_matches_chi = []
for x in rest_matches_chi:
    if x not in unique_rest_matches_chi:
        unique_rest_matches_chi.append(x)
len(unique_rest_matches_chi)

43

In [102]:
unique_rest_matches_chi[0:5]

[['los comales'],
 ['la amistad inc.', 'la amistad'],
 ['las palmas'],
 ['la fuente'],
 ['clark el ranchito corp', 'el ranchito']]

### Getting Regions for Matches

In [103]:
# using the DBA Name as the key
restaurant_matches_chi = {key[0]: None for key in unique_rest_matches_chi}

In [106]:
for key, value in restaurant_matches_chi.items():
    split_item = key.split()
    for word in split_item:
        for key1, value1 in demonym_dictionary.items():
            for city in value1:
                if word == city:
                    value_info = [key1, city]
                    restaurant_matches_chi[key] = value_info
len(restaurant_matches_chi)

41

In [107]:
for key, value in restaurant_matches_chi.items():
    if value == None:
        for x in unique_rest_matches_chi:
            if x[0] == key:
                if len(x) > 1:
                    split_item = [x.split() for x in x[1:]]
                    for word in split_item:
                        for key1, value1 in demonym_dictionary.items():
                            for city in value1:
                                if word == city:
                                    value_info = [key1, city]
                                    nyc_mex_rest_w_region[key] = value_info

In [108]:
# not sure why some still equal None....
none_counter = 0
for key, value in restaurant_matches_chi.items():
    if value == None:
        none_counter += 1
none_counter

20

In [109]:
# only 26 are genuine matches
len(restaurant_matches_chi) - none_counter

21

In [110]:
# matching words
matching_words_list = []
for key, value in restaurant_matches_chi.items():
    if value != None:
        matching_words_list.append(value[1])

#### Removing values with a value of None

In [111]:
bad_keys_chi = []
for key, value in restaurant_matches_chi.items():
    if value == None:
        bad_keys_chi.append(key)

#### Removing values that passed the filter incorrectly

In [112]:
for key, value in restaurant_matches_chi.items():
    if value != None:
        if value[1] in bad_matches:
            bad_keys_chi.append(key)

In [113]:
# removing bad keys
for x in bad_keys_chi:
    if restaurant_matches_chi.get(x) != None:
        del restaurant_matches_chi[x]
    elif restaurant_matches_chi.get(x) == None:
        del restaurant_matches_chi[x]

In [114]:
len(restaurant_matches_chi.keys())

6

In [115]:
restaurant_matches_chi

{'chilango': ['Ciudad de Mexico', 'chilango'],
 'los alamos': ['Tabasco', 'alamos'],
 'tecalitlan': ['Baja California', 'tecalitlan'],
 'ocotlan': ['Puebla', 'ocotlan'],
 'uruapan': ['Guerrero', 'uruapan'],
 'teloloapan': ['Guerrero', 'teloloapan']}

#### Correcting some errors in regions

In [116]:
restaurant_matches_chi['ocotlan'][0] = 'Jalisco'

In [117]:
restaurant_matches_chi['tecalitlan'][0] = "Jalisco"

In [118]:
restaurant_matches_chi['uruapan'][0] = "Michoacan"

In [119]:
restaurant_matches_chi

{'chilango': ['Ciudad de Mexico', 'chilango'],
 'los alamos': ['Tabasco', 'alamos'],
 'tecalitlan': ['Jalisco', 'tecalitlan'],
 'ocotlan': ['Jalisco', 'ocotlan'],
 'uruapan': ['Michoacan', 'uruapan'],
 'teloloapan': ['Guerrero', 'teloloapan']}

### Getting regions count

In [120]:
regions_represented_chi = []
for key, value in restaurant_matches_chi.items():
    if value[0] not in regions_represented_chi:
        regions_represented_chi.append(value[0])

In [121]:
regions_represented_chi[0:3]

['Ciudad de Mexico', 'Tabasco', 'Jalisco']

In [122]:
# getting the count of values
region_dict_chi = {key: 0 for key in regions_represented_chi}

In [123]:
for key, value in restaurant_matches_chi.items():
    if value[0] in region_dict_chi.keys():
        region_dict_chi[value[0]] += 1

In [124]:
region_dict_chi

{'Ciudad de Mexico': 1,
 'Tabasco': 1,
 'Jalisco': 2,
 'Michoacan': 1,
 'Guerrero': 1}

In [125]:
chicago_regions_represented = []
for key, value in region_dict_chi.items():
    item = [key, round(value/7, 2)]
    chicago_regions_represented.append(item)

In [126]:
chicago_regions_represented

[['Ciudad de Mexico', 0.14],
 ['Tabasco', 0.14],
 ['Jalisco', 0.29],
 ['Michoacan', 0.14],
 ['Guerrero', 0.14]]

#### Making pct as a dict

In [127]:
chicago_regions_pct = {value[0]:value[1] for value in chicago_regions_represented}

### Getting Values that matches

In [129]:
chi_values = {value[1]:0 for key, value in restaurant_matches_chi.items()}

In [130]:
for key, value in restaurant_matches_chi.items():
    if value[1] in chi_values.keys():
        chi_values[value[1]] += 1

In [131]:
# https://careerkarma.com/blog/python-sort-a-dictionary-by-value/
sort_values_chi = sorted(chi_values.items(), key=lambda x: x[1], reverse=True)
for i in sort_values_chi[0:5]:
    print(i[0], i[1])

chilango 1
alamos 1
tecalitlan 1
ocotlan 1
uruapan 1


## Dallas

In [153]:
dallas_data = pd.read_csv("./food_inspections_data/Dallas_inspections_October_2016_to_Present_.csv")
dallas_data.shape

(45844, 114)

In [154]:
dallas_data.columns

Index(['Restaurant Name', 'Inspection Type', 'Inspection Date',
       'Inspection Score', 'Street Number', 'Street Name', 'Street Direction',
       'Street Type', 'Street Unit', 'Street Address',
       ...
       'Violation Points - 24', 'Violation Detail - 24', 'Violation Memo - 24',
       'Violation Description - 25', 'Violation Points - 25',
       'Violation Detail - 25', 'Violation Memo - 25', 'Inspection Month',
       'Inspection Year', 'Lat Long Location'],
      dtype='object', length=114)

### Only unique dallas restaurants

In [155]:
unique_dallas_data = dallas_data.drop_duplicates(subset=['Restaurant Name'], keep='first')

In [156]:
unique_dallas_data.shape

(7474, 114)

In [157]:
# about an 84% reduction in size of df
unique_dallas_data.shape[0] / dallas_data.shape[0]

0.16303114911438793

### Removing columns from data

In [158]:
# most of the columns relate to inspections
col_names = list(unique_dallas_data.columns)

In [159]:
col_names[0:11]

['Restaurant Name',
 'Inspection Type',
 'Inspection Date',
 'Inspection Score',
 'Street Number',
 'Street Name',
 'Street Direction',
 'Street Type',
 'Street Unit',
 'Street Address',
 'Zip Code']

In [160]:
col_names[-3:]
unique_dallas_data.drop(columns = [x for x in col_names[11:-4]], inplace=True)

In [161]:
unique_dallas_data.head(2)

,Restaurant Name,Inspection Type,Inspection Date,Inspection Score,Street Number,Street Name,Street Direction,Street Type,Street Unit,Street Address,Zip Code,Violation Memo - 25,Inspection Month,Inspection Year,Lat Long Location
0,FRESHII,Routine,10/31/2018,96,2414,VICTORY PARK,NaN,LN,NaN,2414 VICTORY PARK LN,75219,NaN,Oct 2018,FY2019,"2414 VICTORY PARK LN\n(32.787625, -96.809294)"
1,MICKLE CHICKEN,Routine,10/30/2019,100,3203,CAMP WISDOM,W,RD,NaN,3203 W CAMP WISDOM RD,75237,NaN,Oct 2019,FY2020,"3203 W CAMP WISDOM RD\n(32.662584, -96.873446)"


### Getting matches to dem dict

In [162]:
# using the DBA Name as the key
restaurant_matches_dal = {key.lower(): None for key in unique_dallas_data['Restaurant Name'] if type(key) == str}

In [163]:
for key, value in restaurant_matches_dal.items():
    if type(key) == str:
        split_item = key.split()
        for word in split_item:
            for key1, value1 in demonym_dictionary.items():
                for city in value1:
                    if word.lower() == city:
                        value_info = [key1, city]
                        restaurant_matches_dal[key] = value_info

In [164]:
bad_keys_dal = []
for key, value in restaurant_matches_dal.items():
    if value == None:
        bad_keys_dal.append(key)
len(bad_keys_dal)

7084

In [165]:
len(restaurant_matches_dal)

7472

In [166]:
list(restaurant_matches_dal.items())[0:3]

[('freshii', None),
 ('mickle chicken', None),
 ('world trade center market', None)]

### Removing restaurants with no matches

In [167]:
for x in bad_keys_dal:
    if restaurant_matches_dal.get(x) != None:
        del restaurant_matches_dal[x]
    elif restaurant_matches_dal.get(x) == None:
        del restaurant_matches_dal[x]
len(list(restaurant_matches_dal.items()))

388

### Removing Bad Matches

In [168]:
bad_keys_dal2 = []
for key, value in restaurant_matches_dal.items():
    if value != None:
        if value[1] in bad_matches:
            bad_keys_dal2.append(key)

In [169]:
# removing bad keys
for x in bad_keys_dal2:
    if restaurant_matches_dal.get(x) != None:
        del restaurant_matches_dal[x]
    elif restaurant_matches_dal.get(x) == None:
        del restaurant_matches_dal[x]
len(restaurant_matches_dal)

140

### Getting the number of restaurants for each region

In [170]:
regions_represented_dal = []
for key, value in restaurant_matches_dal.items():
    if value[0] not in regions_represented_dal:
        regions_represented_dal.append(value[0])

In [171]:
# getting the count of values
region_dict_dal = {key: 0 for key in regions_represented_dal}
regions_represented_dal[0:4]

['Michoacan de Ocampo', 'Tamaulipas', 'Chihuahua', 'Guerrero']

In [172]:
for key, value in restaurant_matches_dal.items():
    if value[0] in region_dict_dal.keys():
        region_dict_dal[value[0]] += 1
region_dict_dal

{'Michoacan de Ocampo': 30,
 'Tamaulipas': 2,
 'Chihuahua': 2,
 'Guerrero': 10,
 'Nayarit': 1,
 'Durango': 2,
 'Oaxaca': 2,
 'Veracruz': 12,
 'Nuevo Leon': 5,
 'Puebla': 1,
 'Yucatan': 6,
 'Campeche': 14,
 'Jalisco': 6,
 'Guanajuato': 2,
 'Morelos': 4,
 'Baja California Sur': 3,
 'Mexico': 7,
 'Quintana Roo': 1,
 'Hidalgo': 8,
 'Baja California': 3,
 'San Luis Potosi': 3,
 'Tabasco': 5,
 'Zacatecas': 1,
 'Tlaxcala': 1,
 'Sinaloa': 3,
 'Ciudad de Mexico': 2,
 'Chiapas': 3,
 'Sonora': 1}

### Getting Counts as percentages

In [173]:
region_dict_dal_pct = {key:round(value/len(restaurant_matches_dal), 2) 
                       for key, value in region_dict_dal.items()}

### Getting Values that matches

In [174]:
dal_values = {value[1]:0 for key, value in restaurant_matches_dal.items()}

In [175]:
for key, value in restaurant_matches_dal.items():
    if value[1] in dal_values.keys():
        dal_values[value[1]] += 1

In [176]:
# https://careerkarma.com/blog/python-sort-a-dictionary-by-value/
sort_values_dal = sorted(dal_values.items(), key=lambda x: x[1], reverse=True)
for i in sort_values_dal[0:5]:
    print(i[0], i[1])

michoacana 28
monterrey 11
regio 5
jalisco 5
rodeo 4


## San Francisco 

In [180]:
sf_data = pd.read_csv("./food_inspections_data/san_fran_Restaurant_Scores_-_LIVES_Standard.csv")
sf_data.shape

(53973, 23)

In [181]:
sf_data.head(2)

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,...,inspection_type,violation_id,violation_description,risk_category,Neighborhoods (old),Police Districts,Supervisor Districts,Fire Prevention Districts,Zip Codes,Analysis Neighborhoods
0,69618,Fancy Wheatfield Bakery,1362 Stockton St,San Francisco,CA,94133,NaN,NaN,NaN,NaN,...,Complaint,69618_20190304_103130,Inadequate sewage or wastewater disposal,Moderate Risk,NaN,NaN,NaN,NaN,NaN,NaN
1,97975,BREADBELLY,1408 Clement St,San Francisco,CA,94118,NaN,NaN,NaN,1.415724e+10,...,Routine - Unscheduled,97975_20190725_103124,Inadequately cleaned or sanitized food contact...,Moderate Risk,NaN,NaN,NaN,NaN,NaN,NaN


In [182]:
sf_data_unique = sf_data.drop_duplicates(subset=['business_name'], keep='first')
sf_data_unique.shape

(5775, 23)

In [183]:
lower_biz_name = [x.lower() for x in sf_data['business_name']]
# this is slightly smaller than the set of unique business names without lowering all of the characters (by 1000)
len(set(lower_biz_name)) 

5672

### Getting Names of Restaurants

In [184]:
rest_matches_sf = []
for item in set(lower_biz_name):
    split_item = item.split()
    for word in split_item:
        for key, value in demonym_dictionary.items():
            for city in value:
                if word == city:
                    if word not in bad_matches: # if not, the number of results is tripled
                        rest_matches_sf.append(item)

In [185]:
unique_sf_matches = list(set(rest_matches_sf))
len(unique_sf_matches)

46

In [186]:
unique_sf_matches[0:3]

['la loma produce #8',
 'california pacific medical center',
 'california grocery']

### Getting Regions for Matches

In [187]:
restaurant_matches_sf = {key: None for key in set(unique_sf_matches)}

In [188]:
for key, value in restaurant_matches_sf.items():
    split_item = key.split()
    for word in split_item:
        for key1, value1 in demonym_dictionary.items():
            for city in value1:
                if word == city:
                    value_info = [key1, city]
                    restaurant_matches_sf[key] = value_info
len(restaurant_matches_sf)

46

#### Removing all entries that passed filter incorrectly

In [189]:
bad_keys = []
for key, value in restaurant_matches_nyc.items():
    if value[1] in bad_matches:
        bad_keys.append(key)
len(bad_keys)

0

In [190]:
for x in bad_keys:
    if restaurant_matches_sf.get(x) != None:
        del restaurant_matches_sf[x]

In [191]:
if restaurant_matches_sf.get('guadalupe elementary school'):
    del restaurant_matches_sf['guadalupe elementary school'] # probably refers to a neighborhood, not something in mex.

In [192]:
len(restaurant_matches_sf)

45

### Getting Region Counts

In [194]:
region_dict_sf = {value[0]: 0 for key, value in restaurant_matches_sf.items()}

In [195]:
for key, value in restaurant_matches_sf.items():
    if value[0] in region_dict_sf.keys():
        region_dict_sf[value[0]] += 1

### Getting Region Counts as Percentages

In [197]:
region_dict_sf_pct = {value[0]: 0 for key, value in restaurant_matches_sf.items()}

In [198]:
for key, value in region_dict_sf.items():
    region_dict_sf_pct[key] = round(value/len(restaurant_matches_sf), 2)

In [199]:
region_dict_sf_pct

{'Yucatan': 0.11,
 'Guanajuato': 0.4,
 'Chihuahua': 0.07,
 'Baja California': 0.04,
 'Tlaxcala': 0.02,
 'Campeche': 0.04,
 'Guerrero': 0.04,
 'Mexico': 0.07,
 'Tabasco': 0.04,
 'Durango': 0.04,
 'Jalisco': 0.02,
 'Veracruz': 0.04,
 'Oaxaca': 0.02,
 'Chiapas': 0.02}

### Getting Values that matches

In [200]:
sf_values = {value[1]:0 for key, value in restaurant_matches_sf.items()}

In [201]:
for key, value in restaurant_matches_sf.items():
    if value[1] in sf_values.keys():
        sf_values[value[1]] += 1

In [202]:
# https://careerkarma.com/blog/python-sort-a-dictionary-by-value/
sort_values_sf = sorted(sf_values.items(), key=lambda x: x[1], reverse=True)
for i in sort_values_sf[0:5]:
    print(i[0], i[1])

california 18
loma 3
guadalajara 2
monterey 2
guerrero 2


# Combining All Restaurant Matches into One df

In [203]:
# rest_df has all of the NYC results

## dict data

In [204]:
chi_reg_w_pct = pd.DataFrame.from_dict(restaurant_matches_chi, orient='index', columns = ['Region', 'Match'])
chi_reg_w_pct.sort_index(axis=0, inplace=True)
chi_reg_w_pct['City'] = 'Chicago'

dal_reg_w_pct= pd.DataFrame.from_dict(restaurant_matches_dal, orient='index', columns = ['Region', 'Match'])
dal_reg_w_pct.sort_index(axis=0, inplace=True)
dal_reg_w_pct['City'] = 'Dallas'

sf_reg_w_pct= pd.DataFrame.from_dict(restaurant_matches_sf, orient='index', columns = ['Region', 'Match'])
sf_reg_w_pct.sort_index(axis=0, inplace=True)
sf_reg_w_pct['City'] = 'SF'

all_rest_df = pd.concat([chi_reg_w_pct, dal_reg_w_pct, sf_reg_w_pct], sort=True)

In [205]:
all_rest_df.head(2)

,City,Match,Region
chilango,Chicago,chilango,Ciudad de Mexico
los alamos,Chicago,alamos,Tabasco


## NYC data

In [206]:
rest_df['City'] = 'NYC'

In [207]:
rest_df.head()

,Restaurant_name,Region,Matching_word_w_region,City
0,la guadalupe restaurant,Chihuahua,guadalupe,NYC
1,tacos cuautla morales,Veracruz,morales,NYC
2,acapulco deli & restaurant,Tamaulipas,acapulco,NYC
3,rinconcito tepeyac,Colima,tepeyac,NYC
4,tulcingo restaurant,Puebla,tulcingo,NYC


In [208]:
rest_df.set_index(keys=['Restaurant_name'], inplace=True)

In [209]:
rest_df.columns = ['Region', 'Match', 'City']

In [210]:
rest_df.columns

Index(['Region', 'Match', 'City'], dtype='object')

## Merging the two dfs

In [211]:
all_restaurants = pd.concat([all_rest_df,rest_df])

## Saving df

In [1376]:
# all_restaurants.to_csv("./my_saved_data/Final_restaurant_list.csv")

# combining all pct results to a df 

- nyc_region_counts_dict_pct
- chicago_regions_pct
- region_dict_dal_pct
- region_dict_sf_pct

### Making them all have the same length

In [219]:
regions_used = []

In [220]:
def track_regions_found(dictionary, regions_used):
    for key, value in dictionary.items():
        if key not in regions_used:
            regions_used.append(key)
    return regions_used

In [221]:
regions_used1 = track_regions_found(nyc_region_counts_dict_pct, regions_used)
regions_used2 = track_regions_found(chicago_regions_pct, regions_used1)
regions_used3 = track_regions_found(region_dict_dal_pct, regions_used2)
regions_used4 = track_regions_found(region_dict_sf_pct, regions_used3)

In [222]:
# all but one region was found in these datasets (well 2 since michoacan was counted twice)
len(regions_used4)

31

In [223]:
def add_keys_to_dict(dict_w_pct, regions_used = regions_used4):
    for region in regions_used:
        if region not in dict_w_pct.keys():
            dict_w_pct[region] = 0
    return dict_w_pct

In [224]:
nyc_reg_w_pct = add_keys_to_dict(nyc_region_counts_dict_pct, regions_used4)
chi_reg_w_pct = add_keys_to_dict(chicago_regions_pct, regions_used4)
dal_reg_w_pct = add_keys_to_dict(region_dict_dal_pct)
sf_reg_w_pct = add_keys_to_dict(region_dict_sf_pct)

### Making into df

In [225]:
nyc_reg_w_pct_df = pd.DataFrame.from_dict(nyc_reg_w_pct, orient='index', columns = ['NYC'])
nyc_reg_w_pct_df.sort_index(axis=0, inplace=True)

chi_reg_w_pct_df = pd.DataFrame.from_dict(chi_reg_w_pct, orient='index', columns = ['Chi'])
chi_reg_w_pct_df.sort_index(axis=0, inplace=True)

dal_reg_w_pct_df = pd.DataFrame.from_dict(dal_reg_w_pct, orient='index', columns = ['Dal'])
dal_reg_w_pct_df.sort_index(axis=0, inplace=True)

sf_reg_w_pct_df = pd.DataFrame.from_dict(sf_reg_w_pct, orient='index', columns = ['SF'])
sf_reg_w_pct_df.sort_index(axis=0, inplace=True)

In [226]:
reg_rep_df = pd.concat([nyc_reg_w_pct_df, chi_reg_w_pct_df, dal_reg_w_pct_df, sf_reg_w_pct_df], sort=True, axis=1)

#### Fixing 2 Misnamed columns

In [227]:
reg_rep_df.reset_index(inplace=True)

In [228]:
reg_rep_df['Chi'][15] = reg_rep_df['Chi'][14]

In [229]:
reg_rep_df.drop(index=14, inplace=True)

In [230]:
reg_rep_df['Chi'][20] = reg_rep_df['Chi'][21]

In [231]:
reg_rep_df.drop(index=21, inplace=True)

### Getting the count of the number of restaurants for each column

In [232]:
rest_count_dict = {}
for col in reg_rep_df.columns[1:]:
    counter = 0
    for item in reg_rep_df[col]:
        if item > 0:
            counter += 1
    rest_count_dict[col] = counter   

In [233]:
# despite all cities having more than 1.3 million people, 
# there is a vastly different number of mex. restaurants in each that have names referring to a city or region in mex.
rest_count_dict

{'NYC': 20, 'Chi': 5, 'Dal': 27, 'SF': 14}

#### Saving restaurant count dict

In [235]:
# with open ('restaurant_counts.pickle', 'wb+') as f:
#     pickle.dump(rest_count_dict, f)

# Percent Matches for Each City

In [236]:
# how many unique mexican restaurants were there for each city and how many were matched in the end

In [248]:
num_nyc_restaurants = 0
for key, value in nyc_region_counts_dict.items():
    num_nyc_restaurants += value
num_nyc_restaurants

86

In [249]:
num_nyc_restaurants / unique_nyc_mex.shape[0]
# chicago, dallas SF did not sort restaurants by cuisine type 

0.10129564193168433

# total count df for all cities combined

In [257]:
all_restaurants_cities_dict = {}
for dictionary in [nyc_region_counts_dict, region_dict_chi, region_dict_dal, region_dict_sf]:
    for key, value in dictionary.items():
        if key not in all_restaurants_cities_dict.keys():
            all_restaurants_cities_dict[key] = value
        elif key in all_restaurants_cities_dict.keys():
            all_restaurants_cities_dict[key] += value

In [282]:
# moving michoacan to the key Michoacan de Ocampo
all_restaurants_cities_dict['Michoacan de Ocampo'] += all_restaurants_cities_dict['Michoacan']
del all_restaurants_cities_dict['Michoacan']

In [286]:
all_rest_df = pd.DataFrame.from_dict(all_restaurants_cities_dict, orient='index', columns = ['Count_of_restaurants'])


In [287]:
all_rest_df.Count_of_restaurants.sum()

277

In [288]:
all_rest_df['Percent_of_all_restaurants'] = round(all_rest_df['Count_of_restaurants'] / \
                                                  all_rest_df.Count_of_restaurants.sum(), 2)

In [289]:
all_rest_df.sort_values(by = ['Percent_of_all_restaurants'], ascending=False).head()

,Count_of_restaurants,Percent_of_all_restaurants
Puebla,33,0.12
Michoacan de Ocampo,31,0.11
Yucatan,22,0.08
Guanajuato,20,0.07
Veracruz,17,0.06


In [290]:
# all_rest_df.to_csv("./my_saved_data/all_restaurants_df.csv")

# Conclusions

All 4 cities have large Mexican immigrant populations (250k and above according to: https://247wallst.com/economy/2017/01/27/us-cities-with-the-most-mexican-immigrants/). Despite this surface similarity, it turns out that these immigrants do not all come from similar regions in Mexico. No region was in the top 5 most popular region for more than 2 of these cities. This suggests that immigration from Mexico was not dominated by one region but came from all regions of Mexico. Similarly, we see that there are no single region that makes up the majority of any US city's Mexican population: in all cases, no region accounted for more than 36% of the restaurants with regions associated with their names. This did have some variation, however: in San Francisco Guanajuato accounts for 40% of the restaurants while in Dallas the most dominant group accounts for 21%.

Some shortcomings of this project are the quality of this data: both for the demonym dictionary and the inspection data. The demonym dictionary had many entries, but there were some clear errors, e.g.: the restaurant 'jalisco' in Dallas was associated with "Ciudad de Mexico" instead of the region Jalisco. Additionally, it is not clear if **all** restaurants in each city are included in this data, or only restaurants that failed the inspection. In a similar vein, some cities had data that had too low of a quality to be useful (LA). Another problem with this project is deciding when a "match" is legitimate. For example, "Pueblito" ("little town") appeared often and is a town in Queretáro, but do these restaurants refer to the former or the latter? This was decided on an ad hoc basis and was founded solely on my (non-native) discretion. Lastly, the obvious issue of immediate representation being a marker of the true number of people from a certain region is certainly flawed. Just because there are say, a lot of Italian restaurants in a certain area, does not necessarily mean that there are many italian (recent) immigrants to that area. The popularity of food from certain regions may skew the data. 

Anecdotally, it appears that chain migration happens at a micro level. "Tulcingo" appears 5 times in the NYC data despite the fact that the town that it refers to in Mexico, Tulcingo del Valle, has only 9,000 people living in it (source: https://es.wikipedia.org/wiki/Municipio_de_Tulcingo)! 